In [1]:
import queue as q
import copy as cp
from queue import PriorityQueue

In [2]:
#this is essentially the setting up of the board:{remember, the rules for where the numbers can be moved is not 
#given, it is in the descretion of the user enter the numbers right and do the right actions}

class Puzzle8:
    
    node_num = 1
    max_fringe_len = 1
    explored = []

        
    def __eq__(self,other):
        a1 = (self.number_pos[1] == other.number_pos[1])
        a2 = (self.number_pos[2] == other.number_pos[2])
        a3 = (self.number_pos[3] == other.number_pos[3])
        a4 = (self.number_pos[4] == other.number_pos[4])
        a5 = (self.number_pos[5] == other.number_pos[5])
        a6 = (self.number_pos[6] == other.number_pos[6])
        a7 = (self.number_pos[7] == other.number_pos[7])
        a8 = (self.number_pos[8] == other.number_pos[8])
        a9 = (self.number_pos[9] == other.number_pos[9])
        return (a1 and a2 and a3 and a4 and a5 and a6 and a7 and a8 and a9)
    
    def __lt__(self, other):
        return False
    

    
    def possible_moves(self):        
        for i in range(1,10):
          if(self.number_pos[i] == 'emt'):
            emt_pos = i
            break

        if(emt_pos == 1):
            return ['l','u']
        if(emt_pos == 2):
            return ['r','l','u']
        if(emt_pos == 3):
            return ['r','u']
        if(emt_pos == 4):
            return ['l','u','d']
        if(emt_pos == 5):
            return ['r','l','u','d']
        if(emt_pos == 6):
            return ['r','u','d']
        if(emt_pos == 7):
            return ['l','d']
        if(emt_pos == 8):
            return ['r','l','d']
        if(emt_pos == 9):
            return ['r','d']
    
    def mov_left(self):
        for i in range(1,10):
          if(self.number_pos[i] == 'emt'):
            emt_pos = i
        self.number_pos[emt_pos] = self.number_pos[emt_pos+1]
        self.number_pos[emt_pos+1] = 'emt'
        
    def mov_right(self):
        for i in range(1,10):
          if(self.number_pos[i] == 'emt'):
            emt_pos = i
        self.number_pos[emt_pos] = self.number_pos[emt_pos-1]
        self.number_pos[emt_pos-1] = 'emt'
    
    def mov_up(self):
        for i in range(1,10):
          if(self.number_pos[i] == 'emt'):
            emt_pos = i
        self.number_pos[emt_pos] = self.number_pos[emt_pos+3]
        self.number_pos[emt_pos+3] = 'emt'
    
    def mov_down(self):
        for i in range(1,10):
          if(self.number_pos[i] == 'emt'):
            emt_pos = i
        self.number_pos[emt_pos] = self.number_pos[emt_pos-3]
        self.number_pos[emt_pos-3] = 'emt'
        
    def show_board(self):
        print(self.number_pos[1],self.number_pos[2],self.number_pos[3])
        print(self.number_pos[4],self.number_pos[5],self.number_pos[6])
        print(self.number_pos[7],self.number_pos[8],self.number_pos[9])
    
    def get_board(self):
        res = self.number_pos.copy()
        return res
    
    def exist(self):
        return self.number_pos in Puzzle8.explored
    
    #number of misplaced tiles(mismatch count)
    def heur1(self):
        h1 = -1
        for i in range(1,10):
            if(self.number_pos[i] != i):
                h1 += 1
        return h1
    
    def heur2(self):
        h2 = 0
        for i in range(1,10):
            #print(i)
            if(self.number_pos[i] != 'emt'):
                k = i%3
                if(k == 0):
                    num_pos_h = 3
                    num_pos_v = int(i/3)
                else:
                    num_pos_h = k
                    num_pos_v = int(i/3) + 1
                
                p = self.number_pos[i]%3
                if(p == 0):
                    a_num_pos_h = 3
                    a_num_pos_v = int(self.number_pos[i]/3)
                else:
                    a_num_pos_h = p
                    a_num_pos_v = int(self.number_pos[i]/3) + 1
                man_dist = abs(num_pos_h-a_num_pos_h) + abs(num_pos_v-a_num_pos_v)
                h2 += man_dist
                #print(man_dist)

                
        return h2
    
    def goal_test(self):
        for i in range(1,9):
            if(self.number_pos[i] != i):
                return False
        return True

In [3]:
m = {
    1 : 1,
    2 : 8,
    3 : 2,
    4 : 'emt',
    5 : 4,
    6 : 3,
    7 : 7,
    8 : 6,
    9 : 5
}
k = {
    1 : 7,
    2 : 1,
    3 : 5,
    4 : 3,
    5 : 8,
    6 : 4,
    7 : 2,
    8 : 'emt',
    9 : 6
}
l = {
    1 : 7,
    2 : 1,
    3 : 5,
    4 : 3,
    5 : 8,
    6 : 4,
    7 : 2,
    8 : 'emt',
    9 : 6
}
print(k==l)

True


In [17]:
def expand(S):
    pos_moves = S.possible_moves()
    children_nodes = []
 
    for i in range(pos_moves.__len__()):
        if pos_moves[i] == 'r':
            child = Puzzle8()
            child.number_pos = S.get_board()
            child.depth = S.depth + 1
            child.parent = S
            child.mov_right()
            if not(child.exist()):
                Puzzle8.node_num += 1
                Puzzle8.explored.append(child.get_board())
                children_nodes.append(child)
        elif pos_moves[i] == 'l':
            child = Puzzle8()
            child.number_pos = S.get_board()
            child.depth = S.depth + 1
            child.parent = S
            child.mov_left()
            if not(child.exist()):
                Puzzle8.node_num += 1
                Puzzle8.explored.append(child.get_board())
                children_nodes.append(child)
        elif pos_moves[i] == 'u':
            child = Puzzle8()
            child.number_pos = S.get_board()
            child.depth = S.depth + 1
            child.parent = S
            child.mov_up()
            if not(child.exist()):
                Puzzle8.node_num += 1
                Puzzle8.explored.append(child.get_board())
                children_nodes.append(child)
        else:
            child = Puzzle8()
            child.number_pos = S.get_board()
            child.depth = S.depth + 1
            child.parent = S
            child.mov_down()
            if not(child.exist()):
                Puzzle8.node_num += 1
                Puzzle8.explored.append(child.get_board())
                children_nodes.append(child)
    return children_nodes

def make_fifo_queue(lis):
    result = q.Queue()
    for i in range(lis.__len__()):
        result.put(lis[i])
    
    return result


def mod_exp(S,depth):
    temp3_1 = []
    child_list = expand(S)
    temp1 = make_fifo_queue(child_list)
    temp2 = make_fifo_queue(child_list)
    d = depth - 1
    while d > 0:
        while temp2.qsize() != 0:
            C = temp2.get()
            temp3_1 = expand(C)
            temp3_2 = make_fifo_queue(temp3_1)
            while temp3_2.qsize() !=0:
                temp1.put(temp3_2.get())
        d -= 1
        temp2 = make_fifo_queue(temp3_1)
    return temp1
        


def mod_A(initial_state,heur,depth=1):
    root = Puzzle8()
    root.number_pos = initial_state
    root.depth = 0
    root.parent = None
    Puzzle8.explored.append(initial_state)
    fringe = PriorityQueue()
    S = root
    f = 0
    while not(S.goal_test()):
        temp = mod_exp(S,depth)
        while temp.qsize() != 0:
            l = temp.get()
            if heur == 1:
                val = (l.heur1()+l.depth)
                fringe.put((val,l))
            else:
                fringe.put(((l.heur2()+l.depth),l))
                
        Puzzle8.max_fringe_len = max(Puzzle8.max_fringe_len,fringe.qsize())
        if(fringe.qsize() != 0):
            S = fringe.get()[1]
    return S
        
    

In [30]:
Puzzle8.node_num = 1
Puzzle8.max_fringe_len = 1
Puzzle8.explored = []
k1 = mod_A(k,1,10)

In [31]:
k1.depth

19

In [32]:
print(Puzzle8.node_num)
Puzzle8.node_num = 1

11703


In [43]:
print(1*2*3*4*5*6*7*8*9)

362880


1

In [3]:
print(a.get())
print(a.get())

aa
['qedyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy', 'w']
